In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [2]:
# !head data/mfp-diaries.tsv

## Exploratory Data Analysis

In [3]:
df = pd.read_csv('data/mfp-diaries.tsv',
                  sep='\t',
                  header=None,
                  names=['userId','diary_date','food_entries','daily_goal'])

In [4]:
df.head()

,userId,diary_date,food_entries,daily_goal
0,1,2014-09-14,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"[{""meal"": ""MY food"", ""dishes"": [{""nutritions"":...","{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [5]:
len(df['userId'].unique())

9896

In [6]:
# entry = json.loads(df.iloc[2, 2])
# entry

In [7]:
# entry[0].keys()

## UserId, Entry_Date, Daily_Goal  (Need to Parse Data)

In [8]:
df_goal = df.drop(['food_entries'], axis=1)

In [9]:
df_goal.head()

,userId,diary_date,daily_goal
0,1,2014-09-14,"{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,1,2014-09-15,"{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,1,2014-09-16,"{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,1,2014-09-17,"{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,1,2014-09-18,"{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [24]:
parse_df = df_goal.drop(['userId', 'diary_date'], axis=1)
parse_df.head()

,daily_goal
0,"{""total"": [{""name"": ""Calories"", ""value"": 2924}..."
1,"{""total"": [{""name"": ""Calories"", ""value"": 2430}..."
2,"{""total"": [{""name"": ""Calories"", ""value"": 1862}..."
3,"{""total"": [{""name"": ""Calories"", ""value"": 2251}..."
4,"{""total"": [{""name"": ""Calories"", ""value"": 2001}..."


In [11]:
tg_dict = json.loads(parse_df['daily_goal'][0])
tg_dict

{'total': [{'name': 'Calories', 'value': 2924},
  {'name': 'Carbs', 'value': 340},
  {'name': 'Fat', 'value': 114},
  {'name': 'Protein', 'value': 186},
  {'name': 'Sodium', 'value': 3658},
  {'name': 'Sugar', 'value': 109}],
 'goal': [{'name': 'Calories', 'value': 3173},
  {'name': 'Carbs', 'value': 396},
  {'name': 'Fat', 'value': 105},
  {'name': 'Protein', 'value': 160},
  {'name': 'Sodium', 'value': 2300},
  {'name': 'Sugar', 'value': 119}]}

## Parse Function

In [12]:
"""Creates list of keys and values from the first dictionary, and zips them together"""
def daily_totals(tg_dict):    
    total_cols = [list(tg_dict.keys())[0] + "_" + x['name'].lower() 
                  for x in tg_dict['total']]
    total_values = [y['value'] 
                  for y in tg_dict['total']]

    total_dict = {}
    for col, val in zip(total_cols, total_values):
        total_dict[col] = val
    return total_dict

"""Creates list of keys and values from the second dictionary, and zips them together"""
def daily_goals(tg_dict):
    goal_cols = [list(tg_dict.keys())[1] + "_" + k['name'].lower()
                 for k in tg_dict['goal']]
    goal_values = [i['value']
                 for i in tg_dict['goal']]
    
    goal_dict = {}
    for col, val in zip(goal_cols, goal_values):
        goal_dict[col] = val
    return goal_dict

"""Combines new keys and values for 'Total' and 'Goals' into single dictionary"""
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res

"""Iterrates through entire parsed dataframe and appends merged rows"""
def final_parsed_dict():
    results = []
    for row in parse_df['daily_goal'].values:
        tg_dict = json.loads(row)
        results.append(Merge(daily_goals(tg_dict), daily_totals(tg_dict)))
    return results

In [13]:
f = pd.DataFrame(final_parsed_dict())

In [26]:
df_concat = pd.concat([df_goal, f], axis=1)
final_df = df_concat.drop(['daily_goal'], axis=1)

In [76]:
final_df.head().T

,0,1,2,3,4
userId,1,1,1,1,1
diary_date,2014-09-14,2014-09-15,2014-09-16,2014-09-17,2014-09-18
goal_calcium,NaN,NaN,NaN,NaN,NaN
goal_calories,3173,1572,1832,1685,1597
goal_carbs,396,196,229,210,199
goal_chol,NaN,NaN,NaN,NaN,NaN
goal_fat,105,52,61,56,53
goal_fiber,NaN,NaN,NaN,NaN,NaN
goal_iron,NaN,NaN,NaN,NaN,NaN
goal_mon fat,NaN,NaN,NaN,NaN,NaN


In [77]:
# Create sum column of all nutritional values inputted for the day
final_df['total_sum'] = f[['total_calcium', 'total_calories', 'total_carbs','total_chol', 'total_fat', 
                           'total_fiber', 'total_iron', 'total_mon fat', 'total_ply fat', 'total_potass.', 
                           'total_protein', 'total_sat fat', 'total_sodium', 'total_sugar', 'total_trn fat', 
                           'total_vit a', 'total_vit c']
                          ].apply(np.sum, axis=1)

In [86]:
# Create sum column of all nutritional goal values for the day
final_df['goal_sum'] = f[['goal_calcium', 'goal_calories', 'goal_carbs','goal_chol', 'goal_fat', 
                           'goal_fiber', 'goal_iron', 'goal_mon fat', 'goal_ply fat', 'goal_potass.', 
                           'goal_protein', 'goal_sat fat', 'goal_sodium', 'goal_sugar', 'goal_trn fat', 
                           'goal_vit a', 'goal_vit c']
                          ].apply(np.sum, axis=1)

In [95]:
# final_df['achievement'] = final_df['total_sum'].apply(lambda x: 'succeed' 
#                                                       if x <= 6000 
#                                                       else 'fail')

In [109]:
final_df['achievement'] = np.where(abs((final_df['total_sum'] - final_df['goal_sum']) / final_df['goal_sum']) <= 0.1,
                                       'succeed', 'fail')

In [110]:
final_df.head().T

,0,1,2,3,4
userId,1,1,1,1,1
diary_date,2014-09-14,2014-09-15,2014-09-16,2014-09-17,2014-09-18
goal_calcium,NaN,NaN,NaN,NaN,NaN
goal_calories,3173,1572,1832,1685,1597
goal_carbs,396,196,229,210,199
goal_chol,NaN,NaN,NaN,NaN,NaN
goal_fat,105,52,61,56,53
goal_fiber,NaN,NaN,NaN,NaN,NaN
goal_iron,NaN,NaN,NaN,NaN,NaN
goal_mon fat,NaN,NaN,NaN,NaN,NaN


### Goal:
Create 1 column per dictionary in both total key and goal key

- total
    + total_calories
    + total_carbs
    + total_fat
    + total_protein
    + total_sodium
    + total_sugar
- goal
    + goal_calories
    + goal_carbs
    + goal_fat
    + goal_protein
    + goal_sodium
    + goal_sugar

In [20]:
# json_dict.keys()[0]
# Gives Error: 'dict_keys' object does not support indexing, hence needs to be set to a list to iterate.

In [70]:
df = pd.DataFrame([[1, 4],
                   [2, 5],
                   [3, 6]],
                  columns=["a", "b"])
df

,a,b
0,1,4
1,2,5
2,3,6


In [71]:
df["c"] = df.apply(np.sum, axis=1)
df

,a,b,c
0,1,4,5
1,2,5,7
2,3,6,9
